In [1]:
#https://datahexa.com/kmeans-clustering-with-wine-dataset/参考

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import cluster, preprocessing, datasets

from sklearn.cluster import KMeans

In [8]:
# ワインのデータセットを読み込み
# 説明変数はアルコール濃度や成分など13種類、目的変数は3クラスのワイン品種
wine = datasets.load_wine()

In [9]:
X = wine.data

In [21]:
X[:3]

array([[1.423e+01, 1.710e+00, 2.430e+00, 1.560e+01, 1.270e+02, 2.800e+00,
        3.060e+00, 2.800e-01, 2.290e+00, 5.640e+00, 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, 1.120e+01, 1.000e+02, 2.650e+00,
        2.760e+00, 2.600e-01, 1.280e+00, 4.380e+00, 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, 1.860e+01, 1.010e+02, 2.800e+00,
        3.240e+00, 3.000e-01, 2.810e+00, 5.680e+00, 1.030e+00, 3.170e+00,
        1.185e+03]])

In [11]:
X.shape

(178, 13)

In [12]:
y=wine.target

In [24]:
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
y.shape

(178,)

In [25]:
wine.target_names

array(['class_0', 'class_1', 'class_2'], dtype='<U7')

In [36]:
# k平均法のモデル作成
model = KMeans(n_clusters=3)

In [37]:
# 教師データなしでクラスタリング
labels = model.fit_predict(X)

In [38]:
df = pd.DataFrame({'labels': labels})
type(df)

pandas.core.frame.DataFrame

In [39]:
def species_label(theta):
    if theta == 0:
        return wine.target_names[0]
    if theta == 1:
        return wine.target_names[1]
    if theta == 2:
        return wine.target_names[2]

In [40]:
df['species'] = [species_label(theta) for theta in wine.target]

In [41]:
# 実際のワインの品種['species']と予測したラベル['labels']を比較
pd.crosstab(df['labels'], df['species'])

species,class_0,class_1,class_2
labels,,,
0,13,20,29
1,46,1,0
2,0,50,19


## 考察
- クラスタリングのみを行うため、分類番号とはラベルは関係性がない。
- 全ての説明変数を使うべきか検討する必要がある。

In [124]:
wine_df = pd.DataFrame(wine.data, columns=wine.feature_names)

In [125]:
wine_df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [126]:
wine_df.drop(['alcohol', 'magnesium', 'nonflavanoid_phenols', 'ash', 'malic_acid', 'hue', 'total_phenols', 'color_intensity', 'proline', 'proanthocyanins', 'alcalinity_of_ash', 'od280/od315_of_diluted_wines'], axis=1, inplace=True)
# wine_df.drop(['proline'], axis=1, inplace=True)
wine_df

,flavanoids
0,3.06
1,2.76
2,3.24
3,3.49
4,2.69
...,...
173,0.61
174,0.75
175,0.69
176,0.68


In [127]:
# k平均法のモデル作成
model2 = KMeans(n_clusters=3)
labels2 = model.fit_predict(wine_df)

df2 = pd.DataFrame({'labels': labels2})
df2['species'] = [species_label(theta) for theta in wine.target]

In [128]:
# 実際のワインの品種['species']と予測したラベル['labels']を比較
pd.crosstab(df2['labels'], df2['species'])

species,class_0,class_1,class_2
labels,,,
0,0,11,47
1,50,15,0
2,9,45,1


## 考察
- flavanoidsだけでも3クラス分類ができそうである。
- class_2についてはflavanoidsだけのほうが精度が良い可能性がある。
- 各説明変数を分析し、タスクにあったものを選ぶ必要がある。